In [1]:
import sqlite3

In [2]:
sqlite3.version

'2.6.0'

In [3]:
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

In [5]:
print(type(cur), dir(cur))

<class 'sqlite3.Cursor'> ['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'arraysize', 'close', 'connection', 'description', 'execute', 'executemany', 'executescript', 'fetchall', 'fetchmany', 'fetchone', 'lastrowid', 'row_factory', 'rowcount', 'setinputsizes', 'setoutputsize']


In [4]:
cur.execute('CREATE TABLE people (name_last, age);')

In [8]:
## qmark style
who = 'LJH'; age = 26
cur.execute('INSERT INTO people VALUES(?, ?)', (who, age))

In [13]:
## named style
cur.execute('select * from people where name_last=:who and age=:age',
           {'who':who, 'age':age})

In [14]:
print(cur.fetchone())
# cur.fetchall()

('LJH', 26)


In [19]:
cur.execute('INSERT INTO people VALUES(:a, :b)',
            {'a':who, 'b':age})
print(cur.fetchone())

None


In [22]:
data = [('a', 1),('b', 2),('c', 3),]
cur.executemany('INSERT INTO people VALUES (?,?)', data)

In [30]:
cur.close(); conn.close()

In [31]:
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

In [32]:
cur.executescript('''
    CREATE TABLE people (name, age);
    INSERT INTO people VALUES('a', 1);
''')

In [33]:
cur.execute('SELECT * FROM people;')
cur.fetchall()

[('a', 1)]

In [34]:
cur.close(); conn.close()

In [35]:
conn = sqlite3.connect('ex1.db') ## 
cur = conn.cursor()

In [36]:
cur.executescript('''
    CREATE TABLE people (name, age);
    INSERT INTO people VALUES("a", 1);
''')

cur.execute('SELECT * FROM people;')
cur.fetchall()

[('a', 1)]

In [38]:
cur.execute('INSERT INTO people VALUES("b", 2)')

cur.execute('SELECT * FROM people;')
cur.fetchall()

[('a', 1), ('b', 2)]

In [41]:
cur.executemany('INSERT INTO people VALUES(?, ?)', [("c",3 ), ("d", 4)])

cur.execute('SELECT * FROM people;')
cur.fetchall()

[('a', 1), ('b', 2), ('c', 3), ('d', 4), ('c', 3), ('d', 4)]

In [42]:
# cur.close(); conn.close()

In [43]:
conn = sqlite3.connect('ex1.db') ## 
cur = conn.cursor()

In [44]:
cur.executescript('''
    CREATE TABLE people (name, age);
    INSERT INTO people VALUES("a", 1);
''')

cur.executescript('''
    INSERT INTO people VALUES("E", 5);
    INSERT INTO people VALUES("F", 6);
    INSERT INTO people VALUES("G", 7);
''')

cur.execute('SELECT * FROM people;')
cur.fetchall()

[('a', 1), ('E', 5), ('F', 6), ('G', 7)]

In [45]:
cur.executemany('INSERT INTO people VALUES(?, ?)', [("c",3 ), ("d", 4)])

cur.execute('SELECT * FROM people;')
cur.fetchall()

[('a', 1), ('E', 5), ('F', 6), ('G', 7), ('c', 3), ('d', 4)]

In [46]:
conn.commit() # 실제 물리적 db에 반영해준다

In [49]:
cur.execute('CREATE TABLE temp (aaa, bbb)')

OperationalError: table temp already exists

In [48]:
conn.commit() # 실제 물리적 db에 반영해준다

In [71]:
cur.close(); conn.close()

In [72]:
conn = sqlite3.connect('ex1.db') ## 
cur = conn.cursor()

In [73]:
# cur.executescript('''
#     CREATE TABLE person (
#         first_name text primary key
#         last_name text not null
#     );
    
#     INSERT INTO person VALUES ('name', 'kim');
# ''')

# cur.execute('SELECT * FROM person')
# print(cur.fetchall())

In [74]:
# cur.execute('DROP TABLE IF EXISTS asdfa')

In [75]:
cur.executescript('''
    DROP TABLE IF EXISTS company;
    create table company(
        id INT PRIMARY KEY NOT NULL,
        name TEXT NOT NULL,
        age INT NOT NULL,
        address CHAR(50),
        salary REAL
    );
''')

In [77]:
cur.execute('INSERT INTO company VALUES (1, "AAA", 33, "AB", "1561987646848134846481313546841")')

In [81]:
cur.execute('INSERT INTO company (id, name, age) VALUES(2, "BBB", 3)')

In [82]:
conn.commit() # 실제 물리적 db에 반영해준다

In [83]:
cur.lastrowid, conn.total_changes # 데이터 총 개수, 변화횟수

(2, 2)

In [88]:
## update

# cur.execute('UPDATE company SET name="test";') # 데이터가 test밖에 없어서 전체가 변경된다
cur.execute('UPDATE company SET name="test" WHERE id=1;')
conn.commit()

In [89]:
cur.execute('SELECT * FROM company'); cur.fetchall()

[(1, 'test', 33, 'AB', 1.561987646848135e+30), (2, 'BBB', 3, None, None)]

In [90]:
## commit 안하고 종료 후 열면 자료 날라감!! (warning)
cur.close(); conn.close()

conn = sqlite3.connect('ex1.db') 
cur = conn.cursor()

In [91]:
list(conn.iterdump()) ## 커서가 작업했던 항목들 반환 / 다른 db로 옮겨가야 할 때

['BEGIN TRANSACTION;',
 'CREATE TABLE company(\n        id INT PRIMARY KEY NOT NULL,\n        name TEXT NOT NULL,\n        age INT NOT NULL,\n        address CHAR(50),\n        salary REAL\n    );',
 'INSERT INTO "company" VALUES(1,\'test\',33,\'AB\',1.56198764684813484748e+30);',
 'INSERT INTO "company" VALUES(2,\'BBB\',3,NULL,NULL);',
 'CREATE TABLE people (name, age);',
 'INSERT INTO "people" VALUES(\'a\',1);',
 'INSERT INTO "people" VALUES(\'E\',5);',
 'INSERT INTO "people" VALUES(\'F\',6);',
 'INSERT INTO "people" VALUES(\'G\',7);',
 'INSERT INTO "people" VALUES(\'c\',3);',
 'INSERT INTO "people" VALUES(\'d\',4);',
 'CREATE TABLE temp (aaa, bbb);',
 'COMMIT;']

In [92]:
cur.close(); conn.close()

In [132]:
conn=sqlite3.connect('playlist.db')
cur=conn.cursor()

In [133]:
cur.executescript('''
    DROP TABLE IF EXISTS Artist;
    CREATE TABLE Artist (
        id      INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        title   TEXT NOT NULL
    );
    
    DROP TABLE IF EXISTS Album;
    CREATE TABLE Album (
        id        INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        title     TEXT NOT NULL,
        Artist_id INT NOT NULL
    );
    
    DROP TABLE IF EXISTS Genre;
    CREATE TABLE Genre (
        id        INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        title     TEXT NOT NULL        
    );
    
    DROP TABLE IF EXISTS Track;
    CREATE TABLE Track (
        id        INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        title     TEXT NOT NULL,
        length    INT NOT NULL,
        rating    INT NOT NULL,
        count     INT NOT NULL,
        Album_id  INT NOT NULL,
        Genre_id  INT NOT NULL
    );
''')

In [134]:
cur.execute('INSERT INTO Artist(title) VALUES(?)', ['Led Zepplin'])

In [135]:
cur.lastrowid

1

In [136]:
cur.execute('SELECT id FROM Artist WHERE title=?',['Led Zepplin'])
cur.fetchone()[0]

1

In [137]:
cur.execute('''INSERT INTO Album(title,Artist_id)
VALUES(?,?)''',['IV',cur.lastrowid])

In [138]:
cur.lastrowid

1

In [139]:
cur.execute('''
    INSERT INTO Artist VALUES(NULL, ?)
''', ['AC/DC'])

In [140]:
cur.lastrowid

2

In [141]:
cur.execute('''
    INSERT INTO Album VALUES(NULL, ?, (
        SELECT id FROM Artist WHERE title=? LIMIT 0,1
    ))
''', ['Who Made Who', 'AC/DC'])

In [142]:
cur.lastrowid, conn.total_changes

(2, 4)

In [143]:
conn.commit()

In [144]:
cur.executemany('INSERT INTO Genre(title) VALUES(?)',
                [('Rock',), ('Metal',)])

In [145]:
data = [
    ('Black Dog', 5, 297, 0, 'Who Made Who', 'Rock'),
    ('Stairway', 5, 297, 0, 'Who Made Who', 'Rock'),
    ('About to Rock', 5, 297, 0, 'IV', 'Metal'),
    ('Who Made Who', 5, 297, 0, 'Who Made Who', 'Metal'),
]

cur.executemany('''
    INSERT INTO Track VALUES(NULL, ?, ?, ?, ?,(
        SELECT id FROM Album WHERE title=? LIMIT 0, 1
    ), (
        SELECT id FROM Genre WHERE title=? LIMIT 0, 1
    ))
''', data)

In [146]:
conn.commit()

In [147]:
cur.execute('''
    SELECT * FROM Artist, Album
    WHERE Artist.id=Album.Artist_id
''')
cur.fetchall()

[(1, 'Led Zepplin', 1, 'IV', 1), (2, 'AC/DC', 2, 'Who Made Who', 2)]

In [148]:
cur.execute('''
    SELECT * FROM Album
    INNER JOIN Artist
    ON Artist.id=Album.Artist_id
''')
cur.fetchall()

[(1, 'IV', 1, 1, 'Led Zepplin'), (2, 'Who Made Who', 2, 2, 'AC/DC')]

In [149]:
cur.execute('''
    SELECT * FROM Track AS A
    INNER JOIN Album as B
    ON A.Album_id=B.id
''')
cur.fetchall()

[(1, 'Black Dog', 5, 297, 0, 2, 1, 2, 'Who Made Who', 2),
 (2, 'Stairway', 5, 297, 0, 2, 1, 2, 'Who Made Who', 2),
 (3, 'About to Rock', 5, 297, 0, 1, 2, 1, 'IV', 1),
 (4, 'Who Made Who', 5, 297, 0, 2, 2, 2, 'Who Made Who', 2)]

In [150]:
cur.execute('''
    SELECT A.title, B.title, C.title , D.title FROM Track AS A
    INNER JOIN Album AS B
    ON A.Album_id=B.id
    INNER JOIN Genre AS C
    ON A.Genre_id=C.id
    INNER JOIN Artist AS D
    ON B.Artist_id=D.id
    ORDER BY D.id ASC
    
    ''')
cur.fetchall()

[('About to Rock', 'IV', 'Metal', 'Led Zepplin'),
 ('Black Dog', 'Who Made Who', 'Rock', 'AC/DC'),
 ('Stairway', 'Who Made Who', 'Rock', 'AC/DC'),
 ('Who Made Who', 'Who Made Who', 'Metal', 'AC/DC')]